### work with the netCDF client-side

In [5]:
from app.dataprocessing.local.local_netcdf_reader import LocalNetcdfReader
from app.datastructures.two_dimensional.quad_tree import QuadTree
import matplotlib.pyplot as plt

r = LocalNetcdfReader(2)
tree = QuadTree(max_chunk_size=50, original_file_size=7470,
                dataset=r.get_dataset())


In [6]:
ds, c_bounds = tree.get_initial_dataset()

# use the elevation variable in the plot
init = ds['elevation']

print(init)
print(c_bounds)

plt.imsave('dashboard_render.png', arr=init, origin='lower')


<xarray.DataArray 'elevation' (lat: 3324, lon: 6647)>
[22094628 values with dtype=int16]
Coordinates:
  * lon      (lon) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
  * lat      (lat) float64 -90.0 -89.94 -89.89 -89.84 ... 89.84 89.89 89.94 90.0
Attributes:
    standard_name:       height_above_mean_sea_level
    long_name:           Elevation relative to sea level
    units:               m
    grid_mapping:        crs
    sdn_parameter_urn:   SDN:P01::ALATZZ01
    sdn_parameter_name:  Sea floor height (above mean sea level) {bathymetric...
    sdn_uom_urn:         SDN:P06::ULAA
    sdn_uom_name:        Metres
((-89.99791666666667, 89.99791666666667), (-179.99791666666667, 179.99374999999998))


In [7]:
# ((lat_min, lat_max),(lon_min, lon_max)) to ((sw), (ne))
def convert_bounds(b):
    lat_min = b[0][0]
    lat_max = b[0][1]
    lon_min = b[1][0]
    lon_max = b[1][1]

    res = ((lat_min, lon_min), (lat_max, lon_max))
    return res


#### Render Map

In [ ]:
import ipywidgets
from ipywidgets import Layout
from ipyleaflet import basemaps, Map, ImageOverlay
from datetime import datetime
import xarray as xr

button = ipywidgets.Button(
    description='Render'
)
m = Map(basemap=basemaps.Esri.WorldTopoMap, layout=Layout(width='50%'), zoom = 2)
#m.fit_bounds(convert_bounds(c_bounds))

print(f'bounds:::: {convert_bounds(c_bounds)}')

# init layer
image = ImageOverlay(
    url='dashboard_render.png',
    bounds=convert_bounds(c_bounds),
    opacity=0.4
)
# m.add_layer(image)


output = ipywidgets.Output()
display(button, output, m)


def render(t):
    id = str(datetime.now())[-6:]

    b = m.bounds
    api_bounds = ((b[0][0], b[1][0]), (b[0][1], b[1][1]))
    with output:
        print(f'start new render on: {api_bounds}')

    ds, c_bounds = tree.request_data_m_c(api_bounds, fit_bounds = True)
    with output:
        #print(ds)
        print(f'actual bounds: {c_bounds}')
    ds = ds['elevation']

    with output:
        pass
    plt.imsave(f'dashboard_render{id}.png', arr=ds, origin='lower')

    image = ImageOverlay(
        url=f'dashboard_render{id}.png',
        bounds=convert_bounds(c_bounds),
        opacity=1
    )
    m.add_layer(image)
    with output:
        pass
        #print('render done')


button.on_click(render)
